In [ ]:
from mastodon import Mastodon
from datetime import datetime, timedelta
from kafka import KafkaProducer
import json
import time
import logging
import pytz  # N'oublie pas d'installer pytz si tu ne l'as pas déjà

# 1. Configuration du système de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 2. Configuration de l'accès à l'API Mastodon
logging.info("Configuration de l'accès à l'API Mastodon...")
mastodon = Mastodon(
    access_token='m6d7yGLo1ScqQqfKehJz0YgzuIowJf2UsCK3fRvNBUY',  # Remplacez par votre jeton d'accès
    api_base_url='https://mastodon.social'  # L'URL de votre instance Mastodon
)

# 3. Configuration du producteur Kafka
logging.info("Configuration du producteur Kafka...")
producer = KafkaProducer(
    bootstrap_servers='kafka:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# 4. Calculer la date il y a un mois
one_month_ago = datetime.now(pytz.utc) - timedelta(days=30)  # Assurez-vous que la date soit UTC
logging.info(f"Récupération des toots avant la date : {one_month_ago}")

# 5. Fonction pour récupérer les posts avant une date précise et les envoyer à Kafka
def fetch_mastodon_posts_before_date():
    max_id = None
    while True:
        logging.info("Récupération des toots...")
        # Récupérer les toots avec un paramètre max_id pour la pagination
        toots = mastodon.timeline_public(limit=100, max_id=max_id)

        if not toots:
            logging.info("Aucun nouveau toot trouvé, fin de la récupération.")
            break

        for toot in toots:
            # Ici, toot['created_at'] est déjà un objet datetime
            toot_date = toot['created_at']  # Pas besoin de conversion

            # Vérifier si le toot est plus ancien que la date cible
            if toot_date < one_month_ago:
                logging.info(f"Arrêt de la récupération, la date du toot est {toot_date}, avant la limite d'un mois.")
                return  # On arrête la récupération si on a atteint la date cible

            # Extraire les informations du toot
            toot_data = {
                'id': toot['id'],
                'username': toot['account']['username'],
                'display_name': toot['account']['display_name'],
                'content': toot['content'],
                'created_at': toot['created_at'].isoformat(),  # Convertir en chaîne pour l'envoi à Kafka
                'favourites_count': toot['favourites_count'],
                'reblogs_count': toot['reblogs_count'],
                'replies_count': toot['replies_count']
            }

            # Envoyer les données à Kafka
            producer.send('mastodon-toots', toot_data)
            logging.info(f"Envoyé à Kafka : toot datant de {toot_date}")

        # Mettre à jour max_id pour la pagination
        max_id = toots[-1]['id'] - 1  # Réduire l'id pour continuer à récupérer les posts plus anciens
        logging.info("Pause de 1 seconde avant de récupérer la prochaine page...")
        time.sleep(1)

# 6. Lancer la récupération des posts et l'envoi à Kafka
fetch_mastodon_posts_before_date()

# 7. Fermer le producteur Kafka à la fin (nécessaire si vous arrêtez le programme)
producer.close()
logging.info("Producteur Kafka fermé.")


2024-10-01 14:31:49,016 - INFO - Configuration de l'accès à l'API Mastodon...
2024-10-01 14:31:49,224 - INFO - Configuration du producteur Kafka...
2024-10-01 14:31:49,230 - INFO - <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connecting> [IPv4 ('172.30.0.4', 9092)]>: connecting to kafka:9092 [('172.30.0.4', 9092) IPv4]
2024-10-01 14:31:49,234 - INFO - Probing node bootstrap-0 broker version
2024-10-01 14:31:49,237 - INFO - <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connecting> [IPv4 ('172.30.0.4', 9092)]>: Connection complete.
2024-10-01 14:31:49,349 - INFO - Broker version identified as 2.5.0
2024-10-01 14:31:49,351 - INFO - Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
2024-10-01 14:31:49,362 - INFO - <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connected> [IPv4 ('172.30.0.4', 9092)]>: Closing connection. 
2024-10-01 14:31:49,374 - INFO - Récupération des toots avant la date : 2024-09-01 14:31:49.373775+00:00
20